In [14]:
import pandas as pd
import re

In [15]:
# Libary to import self-made libaries
import sys # https://stackoverflow.com/questions/4383571/importing-files-from-different-folder
sys.path.insert(1, r'C:\Users\ruw\Desktop\RA\Single-use-vs-multi-use-in-health-care\Brighway\Libaries')
import importlib

In [16]:
# Importing self-made libaries
import standards as s
import Monte_Carlo as MC
import life_cycle_assessment as lc
import LCA_plots as lp
import box_plot as bp

import reload_lib as rl

In [17]:
# Path to where the code is stored
path = r'C:\Users\ruw\Desktop\RA\Single-use-vs-multi-use-in-health-care\Brighway'
# Specifying the LCIA method
lcia_method = 'recipe'
lib = [s, MC, lc, lp, bp]

In [18]:
def break_even_initialization(path, lcia_method, lib):
    
    rl.reload_lib(lib)
    flow_legend, database_name, file_name, sheet_name, save_dir, initialization, file_name_unique, db_type = lc.initilization(path, lcia_method)
    df, plot_x_axis_all, impact_categories = lc.quick_LCIA(initialization, file_name, file_name_unique, sheet_name)
    df_rearranged = lc.rearrange_dataframe_index(df)
    if 'recipe' in lcia_method:
        df_res, df_endpoint = lc.recipe_dataframe_split(df_rearranged)
        plot_x_axis_end = plot_x_axis_all[-3:]
        ic_mid = plot_x_axis_all[:-3] 
        plot_x_axis = []
        for ic in ic_mid:
            # print(ic)
            string = re.findall(r'\((.*?)\)', ic)
            if 'ODPinfinite' in  string[0]:
                string[0] = 'ODP'
            elif '1000' in string[0]:
                string[0] = 'GWP'
            plot_x_axis.append(string[0])

    else:
        df_res = df_rearranged
        plot_x_axis = plot_x_axis_all
    
    
    df_col = [df_res.columns[1]]
    df_GWP = df_res[df_col]

    return database_name, df_GWP, db_type, database_name, flow_legend

In [19]:
database_name, df_GWP, db_type, database_name, flow_legend = break_even_initialization(path, lcia_method, lib)

The chosen project is SU_vs_MU and the chosen database is sterilization
Folder name Results created
Chosen flows:
H200 REC - CONSQ
H200 SU - CONSQ
H400 REC- CONSQ
H400 SU - CONSQ
alubox large + washer - CONSQ
alubox large + wipe - CONSQ
alubox small + washer - CONSQ
alubox small + wipe - CONSQ
Recipe is selected
Initialization is completed
DataFrame with nested lists written to Excel successfully.


In [20]:
amount_of_uses = 513
columns = lc.unique_elements_list(database_name)
case = 'break even'
df_be, ignore = lp.process_categorizing(df_GWP, db_type, database_name, case, flow_legend, columns)


df_be_copy = lp.break_even_orginization(df_be, database_name)


# Split index into small and large based on criteria
small_idx = [idx for idx in df_be_copy.index if '2' in idx or 'AS' in idx]
large_idx = [idx for idx in df_be_copy.index if idx not in small_idx]

# Create empty DataFrames for each scenario
scenarios = {
    'small': pd.DataFrame(0, index=small_idx, columns=df_be_copy.columns, dtype=object),
    'large': pd.DataFrame(0, index=large_idx, columns=df_be_copy.columns, dtype=object)
}



# Fill scenarios with data
for sc_idx, (scenario_name, scenario_df) in enumerate(scenarios.items()):
    scenario_df.update(df_be_copy.loc[scenario_df.index])

    alu_box_use, production = {}, {}

    for idx, row in scenario_df.iterrows(): 
        use, prod = 0, 0
        for col in df_be_copy.columns:
            if ('Autoclave' in col or 'Disinfection' in col) and 'H' not in idx:
                alu_box_use[idx] = row[col] + use
                use += row[col]
            elif 'A' in idx:
                production[idx] = (row[col] + prod) * amount_of_uses
                prod += row[col]
                
            else:
                production[idx] = row[col] + prod
                prod += row[col]


In [21]:
production

{'H4S': 1.0608382466482158,
 'H4R': 0.831334278490655,
 'ALC': 20.0723315614131,
 'ALW': 20.0723315614131}

In [22]:
alu_box_use

{'ALC': 0.7900058652549531, 'ALW': 0.6616775182154091}

In [23]:
proc_keys = [key for key in production.keys()]
proc_keys

['H4S', 'H4R', 'ALC', 'ALW']

In [24]:

ALC_be_R = 0
ALC_be_S = 0
ALW_be_R = 0
ALW_be_S = 0

ALC_be = {}
ALW_be = {}

h4r_old = 0
h4s_old = 0

a, b, c, d  = 0, 0, 0, 0

variable_use = range(1,10000,1)
for use in variable_use:
    h4r = production[proc_keys[0]] * use 
    h4s = production[proc_keys[1]] * use
    alc = (production[proc_keys[2]] * amount_of_uses/use + alu_box_use[proc_keys[2]]*use)
    alw = (production[proc_keys[3]] * amount_of_uses/use + alu_box_use[proc_keys[3]]*use)
    # print(h4r, h4s, alc, alw)
    if alc < h4r:
        if alc > h4r_old and a == 0:
            ALC_be[proc_keys[0]] = use
            a = 1
            print(f'Alu + cabinet requires {use} uses to break even to recycled H400')
    if alc < h4s:
        if alc > h4s_old and b == 0:
            ALC_be[proc_keys[1]] = use
            print(f'Alu + cabinet requires {use} uses to break even to SU H400')
            b = 1
    if alw < h4r and c == 0:
        if alw > h4r_old:
            ALC_be[proc_keys[0]] = use
            print(f'Alu + wipe requires {use} uses to break even to recycled H400')
            c = 1
    if alw < h4s:
        if alw > h4s_old and d == 0:
            ALC_be[proc_keys[1]] = use
            print(f'Alu + wipe requires {use} uses to break even to SU H400')
            d = 1

    h4r_old = h4r
    h4s_old = h4s
            
                      

Alu + wipe requires 161 uses to break even to recycled H400
Alu + cabinet requires 195 uses to break even to recycled H400
Alu + wipe requires 247 uses to break even to SU H400
Alu + cabinet requires 500 uses to break even to SU H400
